In [3]:
import ee
import pandas as pd
import os
import numpy as np
ee.Initialize(project='ee-syvt')


In [2]:
# import ee
# import pandas as pd
# import os
# import numpy as np
# ee.Initialize(project='ee-vutrisyiii')


In [4]:
csv_path = 'output.csv'
df = pd.read_csv(csv_path)

In [5]:
df.head()

# Donwload GEDI

In [12]:
# Download GEDI
grid_500kv = ee.FeatureCollection('projects/ee-vutrisyiii/assets/grid500kv')
grid_220kv = ee.FeatureCollection('projects/ee-vutrisyiii/assets/grid220kv')



In [13]:
def getPolygonGeometryById(id):
    polygonFeature = grid_500kv.filter(ee.Filter.eq('id', id)).first()
    if polygonFeature.getInfo() is not None:
      geometry = polygonFeature.geometry() if polygonFeature else None
      return geometry
    
    polygonFeature = grid_220kv.filter(ee.Filter.eq('id', id)).first()
    if polygonFeature.getInfo() is None:
      return None
    
    geometry = polygonFeature.geometry() if polygonFeature else None
    return geometry 

def qualityMask(im):
    # Apply quality and degrade flags in one updateMask call for better performance
    return im.updateMask(im.select('quality_flag').eq(1).And(im.select('degrade_flag').eq(0)))

def download_gedi(index, id_, month, year, output_folder=None):
    id_ = int(id_)
    year = int(year)
    month = int(month)
    
    startOfMonth = ee.Date.fromYMD(year, month, 1)
    endOfMonth = startOfMonth.advance(1, 'month')

    # Get the grid bounds for the given ID
    gridBounds = getPolygonGeometryById(id_)

    if gridBounds:
        # Apply filterBounds to the ImageCollection immediately to reduce size
        dataset = ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')\
                    .filterDate(startOfMonth, endOfMonth)\
                    .filterBounds(gridBounds)\
                    .map(qualityMask)\
                    .select('rh98')

        # If the dataset is not empty, take the first image and clip it
        image = dataset.first().clip(gridBounds).toFloat()

        # Export the image with desired CRS (EPSG:32648)
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f'GEDI_export_{index:04}_{id_}_{month:02}_{year}',
            scale=10,
            crs='EPSG:32648',
            fileFormat='GeoTIFF',
            folder=output_folder or 'GEDI_500_220kv_exports_1',
            maxPixels=1e13
        )

        task.start()

        print(f"Export GEDI_{index:04}_{id_}_{month:02}_{year} to {output_folder}")

In [20]:
start = 4000
end = 5000

for i in range(5, 10):
    id_, m, y = df['id'][i], df['month'][i], df['year'][i]
    download_gedi(i, id_, m, y, f'GEDI_GEE_L2A_V2')

# Download Sentinel-2

In [16]:
# Download s2

def getCloudMaskMonthlyImage(year, month, region):
  startOfMonth = ee.Date.fromYMD(year, month, 1)
  endOfMonth = startOfMonth.advance(1, 'month')

  # Filter Sentinel-2 image collection
  sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                  .filterBounds(region)\
                  .filterDate(startOfMonth, endOfMonth)\
                  .select(['B1', 'B2', 'B3', 'B4','B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'SCL'])
                  # .select(['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12'])
                  
  # Add CLOUDY_PIXEL_PERCENTAGE as a property
  # def mp_func(image):
  #   cloudiness = ee.Number(image.get('CLOUDY_PIXEL_PERCENTAGE'))
  #   return image.set('cloudiness', cloudiness)
  
  def mask_clouds(image):
    scl = image.select('SCL')
    cloud_mask = scl.neq(5).And(scl.neq(6)).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(11))
    return image.updateMask(cloud_mask)
  
  # sentinel2 = sentinel2.map(mp_func)
  sentinel2 = sentinel2.map(mask_clouds)
  

  # Calculate the median image for the month
  medianImage = sentinel2.median().clip(region)

  return medianImage # Return the collection for further use

def download_s2(index, id_, month, year, output_folder=None):
  id_ = int(id_)
  month = int(month)
  year = int(year)
  
  gridBounds = getPolygonGeometryById(id_)

  if gridBounds is not None:
    images = getCloudMaskMonthlyImage(year, month, gridBounds)
    
    # Set the data type before export
    images = images.toFloat()
    # Export the image with desired CRS (WGS84 in this example)
    task = ee.batch.Export.image.toDrive(
        image=images,
        description=f'S2_export_{index:04}_{id_}_{month:02}_{year}',
        scale=10,
        crs='EPSG:32648', 
        fileFormat='GeoTIFF',
        folder=output_folder or 'S2_500_220kv_exports',
        maxPixels=1e13
    )

    task.start()

    print(f"Export S2_{index:04}_{id_}_{month:02}_{year} to {output_folder}")

In [17]:
start = 4000
end = 5000
for i in range(3):
    id_, m, y = df['id'][i], df['month'][i], df['year'][i]
    # download_s2(i, id_, m, y, f'S2_{i // 200:02}')
    download_s2(i, id_, m, y, f'S2_sample')
    
    